##Rotten Tomatoes Classification

Load Dataset

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset


data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

Load a zero-shot classifier

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Define a function to classify a review

In [ ]:
def classify_review(review_text):
    candidate_labels = ["positive", "negative", "neutral"]
    result = classifier(review_text, candidate_labels)

    return result

Convert dataset splits into Pandas DataFrame

In [ ]:
import pandas as pd
train_data = data['train']
test_data=data['test']
df1 = train_data.to_pandas()
df2 = test_data.to_pandas()
dataset = pd.concat([df1, df2], ignore_index=True)
print(dataset)

                                                   text  label
0     the rock is destined to be the 21st century's ...      1
1     the gorgeously elaborate continuation of " the...      1
2                        effective but too-tepid biopic      1
3     if you sometimes like to go to the movies to h...      1
4     emerges as something rare , an issue movie tha...      1
...                                                 ...    ...
9591  a terrible movie that some people will neverth...      0
9592  there are many definitions of 'time waster' bu...      0
9593  as it stands , crocodile hunter has the hurrie...      0
9594  the thing looks like a made-for-home-video qui...      0
9595  enigma is well-made , but it's just too dry an...      0

[9596 rows x 2 columns]


Classify and print first 5 training reviews

In [ ]:
sample_reviews = df1['text'][:5]
for idx, review in enumerate(sample_reviews):
    print(f"Review {idx + 1}:")
    print(review)
    result = classify_review(review)
    print(f"{result['labels'][0]}")
    print(f"{result['scores']}")
    print("-" * 50)

Review 1:
the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
positive
[0.9652496576309204, 0.017758755013346672, 0.01699155941605568]
--------------------------------------------------
Review 2:
the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .
positive
[0.7428897023200989, 0.18453863263130188, 0.07257171720266342]
--------------------------------------------------
Review 3:
effective but too-tepid biopic
negative
[0.909027636051178, 0.05609999969601631, 0.03487243875861168]
--------------------------------------------------
Review 4:
if you sometimes like to go to the movies to have fun , wasabi is a good place to start .
positive
[0.838749349117279, 0.09070141613483429, 0.0705492123961